In [21]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import model_from_json

In [1]:
import cv2
import os
import glob 


In [19]:
batch_size = 64
num_classes = 10
epochs = 10

In [3]:
df_train = pd.read_csv('train.csv')

In [4]:
df_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_train_l = df_train.iloc[:, 0]
np_train_l = np.array(df_train_l)
np_train_l = np_train_l.reshape(42000,1)

In [6]:
df_train_f = df_train.iloc[:, 1:]
np_train_f = np.array(df_train_f)

print(np_train_f.shape)

(42000, 784)


In [7]:
df_train_f.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X_train, X_val, y_train, y_val = train_test_split(np_train_f, np_train_l, test_size=0.10, random_state=42)

In [9]:
y_train = y_train.reshape(37800,1)
y_val = y_val.reshape(4200,1)

In [79]:
# def cnn_model(features,labels, mode):
#     input_layer = tf.reshape(features, [-1, 28, 28, 1])
#     input_layer = tf.cast(input_layer, tf.float32)
#     conv1 = tf.layers.conv2d(inputs = input_layer, filters = 32, kernel_size = [5,5], strides = [1,1], padding = "same", activation = tf.nn.relu)
#     pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size = [2,2], strides = 2)
#     conv2 = tf.layers.conv2d(inputs = pool1, filters = 64, kernel_size = [5,5], padding = "same", activation = tf.nn.relu)
#     pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size = [2,2], strides = 2)
#     pool2_flat = tf.reshape(pool2, [-1, 7*7*64])
#     dense = tf.layers.dense(inputs = pool2_flat, units = 1024, activation = tf.nn.relu)
#     dropout = tf.layers.dropout(inputs = dense, rate = 0.4, training = mode == tf.estimator.Modekeys.TRAIN)
#     logits = tf.layers.dense(inputs = dropout, units= 10 )
#     predictions = {"classes" : tf.argmax(input = logits, axis = 1), "probabilities" : tf.nn.softmax(inputs = logits, name = "SoftmaxTensor")}
    
#     if mode == tf.estimator.Modekeys.PREDICT:
#         return tf.estimator.EstimatorSpec(mode = mode, predictions = predictions)
    
#     loss = tf.losses.sparse_softmax_cross_entropy(labels= labels, logits = logits)
    
#     if mode == tf.estimator.ModeKeys.TRAIN:
#         optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.001)
#         train_op = optimizer.minimize(loss = loss, global_step = tf.train.get_global_step())
#         return tf.estimator.EstimatorSpec(mode = mode, loss = loss, train_op = train_op)
    
#     eval_metric_ops = {"accuracy" : tf.metrics.accuracy(
#           labels=labels, predictions=predictions["classes"])}
  
#     return tf.estimator.EstimatorSpec(
#       mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [1]:
# def cnn_model_fn(features, labels, mode):
#     """Model function for CNN."""
#       # Input Layer
#     input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

#     # Convolutional Layer #1
#     conv1 = tf.layers.conv2d(
#         inputs=input_layer,
#         filters=32,
#         kernel_size=[5, 5],
#         padding="same",
#         activation=tf.nn.relu)

#     # Pooling Layer #1
#     pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

#     # Convolutional Layer #2 and Pooling Layer #2
#     conv2 = tf.layers.conv2d(
#         inputs=pool1,
#       filters=64,
#       kernel_size=[5, 5],
#       padding="same",
#       activation=tf.nn.relu)
#     pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

#   # Dense Layer
#     pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
#     dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
#     dropout = tf.layers.dropout(
#       inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

#   # Logits Layer
#     logits = tf.layers.dense(inputs=dropout, units=10)

#     predictions = {
#       # Generate predictions (for PREDICT and EVAL mode)
#       "classes": tf.argmax(input=logits, axis=1),
#       # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
#       # `logging_hook`.
#       "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
#   }

#     if mode == tf.estimator.ModeKeys.PREDICT:
#         return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

#   # Calculate Loss (for both TRAIN and EVAL modes)
#     onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
#     loss = tf.losses.softmax_cross_entropy(
#     onehot_labels=onehot_labels, logits=logits)
# #     loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

#   # Configure the Training Op (for TRAIN mode)
#     if mode == tf.estimator.ModeKeys.TRAIN:
#         optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
#         train_op = optimizer.minimize(
#         loss=loss,
#         global_step=tf.train.get_global_step())
#     return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

#   # Add evaluation metrics (for EVAL mode)
#     eval_metric_ops = {
#       "accuracy": tf.metrics.accuracy(
#           labels=labels, predictions=predictions["classes"])}
#     return tf.estimator.EstimatorSpec(
#       mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [92]:
# def main(unused_argv):
# #     train_data = X_train # Returns np.array
# #     train_labels = y_train
# #     eval_data = X_val # Returns np.array
# #     eval_labels = y_val
#     mnist = tf.contrib.learn.datasets.load_dataset("mnist")
#     train_data = mnist.train.images # Returns np.array
#     train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
#     eval_data = mnist.test.images # Returns np.array
#     eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
#     mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model, model_dir="./convnet_mnist")
    
#     tensors_to_log = {"probabilities": "SoftmaxTensor"}
#     logging_hook = tf.train.LoggingTensorHook(
#       tensors=tensors_to_log, every_n_iter=50)
    
#     train_input_fn = tf.estimator.inputs.numpy_input_fn(
#     x={"x": train_data},
#     y=train_labels,
#     batch_size=100,
#     num_epochs=None,
#     shuffle=True)
#     mnist_classifier.train(
#     input_fn=train_input_fn,
#     steps=20000,
#     hooks=[logging_hook])
    
#     eval_input_fn = tf.estimator.inputs.numpy_input_fn(
#     x={"x": eval_data},
#     y=eval_labels,
#     num_epochs=1,
#     shuffle=False)
#     eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
#     print(eval_results)

# #     train_input_fn = tf.estimator.inputs.numpy_input_fn(x= {"x":train_data}, y=train_labels,batch_size=100,num_epochs=None,shuffle=True)
    
# #     mnist_classifier.train(input_fn=train_input_fn,steps=20000,hooks=[logging_hook])
    
# #     eval_input_fn = tf.estimator.inputs.numpy_input_fn(x ={"x":eval_data}, y = eval_labels, num_epochs = 1, shuffle = False)
    
# #     eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
#     print(eval_results)

In [17]:
# # if __name__ == "__main__":
# tf.app.run()

In [10]:
img_rows, img_cols = 28, 28

In [11]:
df_test = pd.read_csv('test.csv')

In [12]:
df_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
X_test = np.array(df_test)
print(np.shape(X_test))

(28000, 784)


In [14]:
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_val = X_val.reshape(X_val.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train= X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [15]:
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_train /= 255
X_val /= 255
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_val.shape[0], 'val samples')

x_train shape: (37800, 28, 28, 1)
37800 train samples
4200 val samples


In [16]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
print(y_train.shape)
print(y_val.shape)

(37800, 10)
(4200, 10)


In [20]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5),activation='relu',padding = 'same',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides = 2))
model.add(Conv2D(64, (5,5), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides = 2))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))
score = model.evaluate(X_val, y_val, verbose=0)
print('Val loss:', score[0])
print('Val accuracy:', score[1])

Train on 37800 samples, validate on 4200 samples
Epoch 1/10
37800/37800 [==============================] - 63s 2ms/step - loss: 0.2176 - acc: 0.9311 - val_loss: 0.0667 - val_acc: 0.9798
Epoch 2/10
37800/37800 [==============================] - 64s 2ms/step - loss: 0.0596 - acc: 0.9814 - val_loss: 0.0519 - val_acc: 0.9831
Epoch 3/10
37800/37800 [==============================] - 64s 2ms/step - loss: 0.0429 - acc: 0.9865 - val_loss: 0.0379 - val_acc: 0.9871
Epoch 4/10
37800/37800 [==============================] - 64s 2ms/step - loss: 0.0344 - acc: 0.9896 - val_loss: 0.0356 - val_acc: 0.9895
Epoch 5/10
37800/37800 [==============================] - 64s 2ms/step - loss: 0.0277 - acc: 0.9908 - val_loss: 0.0353 - val_acc: 0.9893
Epoch 6/10
37800/37800 [==============================] - 64s 2ms/step - loss: 0.0215 - acc: 0.9931 - val_loss: 0.0301 - val_acc: 0.9902
Epoch 7/10
37800/37800 [==============================] - 64s 2ms/step - loss: 0.0190 - acc: 0.9944 - val_loss: 0.0302 - val_acc:

In [22]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [23]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [27]:
if K.image_data_format() == 'channels_first':
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_test= X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [28]:
y_test = model.predict(X_test, batch_size = batch_size)

In [30]:
y_test = np.argmax(y_test, axis = 1)

In [31]:
print(y_test)

[2 0 9 ... 3 9 2]


In [35]:
id_ = []
for i in range(1, 28001):
    id_.append(i)
id_ = np.array(id_)

In [37]:
df = pd.DataFrame({"ImageId": id_, "Label": y_test})
df.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [40]:
subm = df.to_csv('digit.csv')